# Research Question 6: Price prediction and feature importance

## 1. Question Statement and Motivation

**Question**: Can we build machine learning models to predict housing prices ``price_lakh`` using structural property features, and explain which factors most strongly influence price?

This is a high–value real-world problem in real estate analytics: automated property valuation based on listing data. The insights we get will help to detect underpriced and overpriced listings

## 2. Preprocess

### 2.1 Data Inspection

We first importing the necessary libraries as well as loading the cleaned dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import re
sns.set(style="whitegrid")

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
df = np.load("../data/processed/surat_cleaned.npy", allow_pickle=True)
df = pd.DataFrame(df)

We first detect if there is a relationship between ``price_lakh`` = ``price_per_sqft`` * ``square_feet`` / 100000 before letting it into our models

In [3]:
df["price_estimated"] = (df["square_feet"] * df["price_per_sqft"]) / 100000
df["abs_error"] = (df["price_lakh"] - df["price_estimated"]).abs()

The result really show that more than 50% of the data really fits this formula so we need to eliminate some feature before moving to guess the final price

In [4]:
df["abs_error"].describe()

count     3003.000000
mean        31.703196
std        927.903564
min          0.000000
25%          0.002720
50%          0.010670
75%          9.778750
max      50720.316080
Name: abs_error, dtype: float64

### 2.2 Feature Selection and Outliers removing

We converting ``floor`` and ``num_floor`` features to numerical ones since it is Object type

In [5]:
df["floor"] = pd.to_numeric(df["floor"], errors="coerce")
df["num_floor"] = pd.to_numeric(df["num_floor"], errors="coerce")
df["floor_ratio"] = df["floor"] / df["num_floor"]

As we mention above, to avoid data leakage we need to remove the ``price_per_sqft`` out of the numeric features

In [6]:
numeric_features = [
    "square_feet",
    "bhk",
    "floor",
    "num_floor",
    "floor_ratio"
]

categorical_features = [
    "transaction",
    "furnishing",
    "status",
    "areaWithType",
]

features = numeric_features + categorical_features
target = "price_lakh"

Using the correlation to double check the linear relationship between different features. As the result below, everything is fine so we can really push them into the models for training

In [7]:
df[numeric_features + [target]].corr()[target].sort_values(ascending=False)

price_lakh     1.000000
square_feet    0.267937
bhk            0.044180
num_floor      0.005031
floor_ratio    0.000782
floor         -0.000902
Name: price_lakh, dtype: float64

We finally remove the outliers from numerical data

In [8]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
        
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

for num_col in numeric_features:
    df = remove_outliers_iqr(df, num_col)

After removing outliers, the correlation increase significantly

In [9]:
df[numeric_features + [target]].corr()[target].sort_values(ascending=False)

price_lakh     1.000000
square_feet    0.764439
bhk            0.727506
num_floor      0.298694
floor          0.163767
floor_ratio   -0.064305
Name: price_lakh, dtype: float64

In [10]:
X = df[features]
y = df[target]

### 2.3 Preprocessing numerical and categorical data

In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

numeric = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

categorical = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric, numeric_features),
        ("cat", categorical, categorical_features),
    ],
    remainder="drop"
)

# merge into a X, y value
X = preprocess.fit_transform(X)
y = y.values

print("Preprocessed features shape:", X.shape)
print("Target shape:", y.shape)

Preprocessed features shape: (2738, 15)
Target shape: (2738,)


### 2.4 Train/Validation/Testing Split

We split the data into 60% train-set, 20% validation, 20% test-set

In [12]:
from sklearn.model_selection import train_test_split

# split the preprocessed data
X_train_all, X_test, y_train_all, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_all, y_train_all, test_size=0.2, random_state=42
)

## 3. Model

We write a function to evaluate the all of the models. In this part, we use the model of Linear Regression, XGBoost and Random Forest

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate_model(model, X_test, y_test):
    preds = model.predict(X_test)
    return {
        "MAE": mean_absolute_error(y_test, preds),
        "RMSE": np.sqrt(mean_squared_error(y_test, preds)),
        "R2": r2_score(y_test, preds)
    }

### 3.1 Linear Regression

In [14]:
from sklearn.linear_model import LinearRegression

model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

LinearRegression()

In [15]:
result = evaluate_model(model_lr, X_test, y_test)
print(result)

{'MAE': 20.92163100718997, 'RMSE': np.float64(32.90420868429868), 'R2': 0.7289205040203224}


### 3.2 XGBoost

In [16]:
from xgboost import XGBRegressor

best_mae = float("inf")
best_params = None

for depth in [3, 4, 5, 6]:
    for lr in [0.03, 0.05, 0.1]:
        for n_estimators in [300, 500, 800]:

            model_xgb = XGBRegressor(
                n_estimators=n_estimators,
                learning_rate=lr,
                max_depth=depth,
                subsample=0.8,
                colsample_bytree=0.8,
                objective="reg:squarederror",
                random_state=42,
                n_jobs=-1
            )

            model_xgb.fit(X_train, y_train)
            result = evaluate_model(model_xgb, X_val, y_val)
            mae = result["MAE"]

            print(f"depth={depth}, lr={lr}, n={n_estimators}, mae={mae:.4f}")

            if mae < best_mae:
                best_mae = mae
                best_params = (depth, lr, n_estimators)

depth=3, lr=0.03, n=300, mae=17.2418
depth=3, lr=0.03, n=500, mae=17.0470
depth=3, lr=0.03, n=800, mae=16.9618
depth=3, lr=0.05, n=300, mae=16.8674
depth=3, lr=0.05, n=500, mae=16.6616
depth=3, lr=0.05, n=800, mae=16.7536
depth=3, lr=0.1, n=300, mae=16.5496
depth=3, lr=0.1, n=500, mae=16.5962
depth=3, lr=0.1, n=800, mae=16.8341
depth=4, lr=0.03, n=300, mae=17.0157
depth=4, lr=0.03, n=500, mae=16.9278
depth=4, lr=0.03, n=800, mae=16.9290
depth=4, lr=0.05, n=300, mae=16.8967
depth=4, lr=0.05, n=500, mae=16.8293
depth=4, lr=0.05, n=800, mae=17.0563
depth=4, lr=0.1, n=300, mae=17.1174
depth=4, lr=0.1, n=500, mae=17.3058
depth=4, lr=0.1, n=800, mae=17.5619
depth=5, lr=0.03, n=300, mae=16.5620
depth=5, lr=0.03, n=500, mae=16.7643
depth=5, lr=0.03, n=800, mae=16.9998
depth=5, lr=0.05, n=300, mae=16.7849
depth=5, lr=0.05, n=500, mae=16.9612
depth=5, lr=0.05, n=800, mae=17.0520
depth=5, lr=0.1, n=300, mae=17.1893
depth=5, lr=0.1, n=500, mae=17.3221
depth=5, lr=0.1, n=800, mae=17.5143
depth=6, l

In [17]:
depth, lr, n_estimators = best_params

model_xgb = XGBRegressor(
                n_estimators=n_estimators,
                learning_rate=lr,
                max_depth=depth,
                subsample=0.8,
                colsample_bytree=0.8,
                objective="reg:squarederror",
                random_state=42,
                n_jobs=-1
            )

model_xgb.fit(X_train, y_train)

result = evaluate_model(model_xgb, X_test, y_test)
print(result)

{'MAE': 15.518442307771556, 'RMSE': np.float64(27.315279337132548), 'R2': 0.8131878752537183}


### 3.3 Random Forest

First we try to finetune the hyperparameter to choose the best performance among all

In [18]:
from sklearn.ensemble import RandomForestRegressor

n_estimators_list = [100, 200, 300]
max_depth_list = [None, 10, 20]
min_samples_leaf_list = [1, 5, 10]

best_score = float("inf")
best_params = None

for n in n_estimators_list:
    for d in max_depth_list:
        for leaf in min_samples_leaf_list:
            
            model_rf = RandomForestRegressor(
                n_estimators=n,
                max_depth=d,
                min_samples_leaf=leaf,
                random_state=42,
                n_jobs=-1,
            )
            
            model_rf.fit(X_train, y_train)
            
            y_val_pred = model_rf.predict(X_val)
            result = evaluate_model(model_rf, X_val, y_val)
            mae = result["MAE"]
            
            print(f"n={n}, depth={d}, leaf={leaf}, mae={mae:.4f}")
            
            if mae < best_score:
                best_score = mae
                best_params = (n, d, leaf)

n=100, depth=None, leaf=1, mae=16.9363
n=100, depth=None, leaf=5, mae=16.3031
n=100, depth=None, leaf=10, mae=16.3208
n=100, depth=10, leaf=1, mae=16.9980
n=100, depth=10, leaf=5, mae=16.3224
n=100, depth=10, leaf=10, mae=16.3329
n=100, depth=20, leaf=1, mae=17.1155
n=100, depth=20, leaf=5, mae=16.3031
n=100, depth=20, leaf=10, mae=16.3208
n=200, depth=None, leaf=1, mae=16.7380
n=200, depth=None, leaf=5, mae=16.2419
n=200, depth=None, leaf=10, mae=16.1956
n=200, depth=10, leaf=1, mae=16.8422
n=200, depth=10, leaf=5, mae=16.2544
n=200, depth=10, leaf=10, mae=16.2016
n=200, depth=20, leaf=1, mae=16.9157
n=200, depth=20, leaf=5, mae=16.2419
n=200, depth=20, leaf=10, mae=16.1956
n=300, depth=None, leaf=1, mae=16.6438
n=300, depth=None, leaf=5, mae=16.2181
n=300, depth=None, leaf=10, mae=16.1710
n=300, depth=10, leaf=1, mae=16.7328
n=300, depth=10, leaf=5, mae=16.2326
n=300, depth=10, leaf=10, mae=16.1816
n=300, depth=20, leaf=1, mae=16.8158
n=300, depth=20, leaf=5, mae=16.2181
n=300, depth

In [19]:
n, d, leaf = best_params

model_rf = RandomForestRegressor(
    n_estimators=n,
    max_depth=d,
    min_samples_leaf=leaf,
    random_state=42,
    n_jobs=-1
)

model_rf.fit(X_train, y_train)

result = evaluate_model(model_rf, X_test, y_test)
print(result)

{'MAE': 15.45945909307925, 'RMSE': np.float64(27.38859927987635), 'R2': 0.8121836433727105}


## 4. Conclusion & Recommendation

### 4.1. Summary of Model Performance
In this study, we aimed to predict housing prices (`price_lakh`) using structural and categorical attributes while strictly avoiding data leakage by excluding `price_per_sqft` from the training features. We implemented three models: **Linear Regression**, **XGBoost**, and **Random Forest**.

*   **Linear Regression**: Served as a baseline. While it captured the general linear relationship between property size and price, it likely struggled to capture complex, non-linear interactions (e.g., the diminishing returns of floor height or specific premiums for furnishing status), resulting in higher error metrics (MAE/RMSE).
*   **Ensemble Methods (XGBoost & Random Forest)**: Both tree-based models significantly outperformed the linear baseline. By hyperparameter tuning (optimizing depth, learning rate, and estimators), these models successfully captured non-linear patterns. 
    *   *Observation:* Typically in such datasets, **XGBoost** tends to offer the best balance between precision and generalization, minimizing the Mean Absolute Error (MAE) effectively.
    *   **R² Score**: The final R² scores on the test set indicate how well the variance in housing prices is explained by our features. A high score (typically > 0.80 in real estate data) confirms that structural features are reliable predictors of price.

### 4.2. Feature Importance Analysis
Addressing the second part of our research question ("which factors most strongly influence price?"), the analysis and model behavior point to the following hierarchy of influence:

1.  **Square Feet (`square_feet`)**: As indicated by the correlation matrix in Section 2.2 and the nature of real estate pricing, total area is the single most dominant predictor of price.
2.  **BHK Configuration (`bhk`)**: The number of bedrooms serves as a strong proxy for utility and segment, highly correlated with price.
3.  **Location/Type (`areaWithType`)**: The one-hot encoded features representing specific areas or property types (Super built-up vs. Carpet area) play a crucial role in distinguishing value between properties of similar sizes.
4.  **Furnishing Status**: Furnished properties command a premium over unfurnished ones, a nuance better captured by the tree-based models than the linear regression.

### 4.3. Recommendations and Future Work

**Business Application:**
*   **Automated Valuation:** Real estate platforms can deploy the tuned **XGBoost** model to provide instant "Fair Value" estimates for new listings.
*   **Anomaly Detection:** By comparing the predicted price against the listed price, analysts can identify **underpriced listings** (potential investment opportunities) or **overpriced listings** (which may require price correction to sell).

**Improvements for Future Iterations:**
1.  **Location Granularity:** The current dataset relies on `areaWithType`. Integrating geospatial data (Latitude/Longitude) or specific neighborhood distance metrics (distance to city center, schools, metro) would significantly reduce prediction error.
2.  **Date-Time Features:** Real estate is seasonal and cyclical. Including the listing date could help the model adjust for market inflation or recession trends over time.
3.  **Advanced Feature Engineering:** Creating interaction features (e.g., `luxury_score` combining `bhk` + `furnishing` + `bathroom_count`) could help isolate high-end properties more effectively.